In [2]:
!pip install fpdf plotly

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
from fpdf import FPDF
import os

# 1. Create sample CSV
def create_sample_csv(filename="transactions.csv"):
    data = {
        'Sender': ['Alice', 'Bob', 'Alice', 'Charlie', 'David', 'Eve', 'Bob', 'Frank', 'Grace', 'Heidi'],
        'Receiver': ['Bob', 'Charlie', 'Charlie', 'Alice', 'Eve', 'Frank', 'Grace', 'Heidi', 'Alice', 'Bob'],
        'Amount': [1200, 1500, 800, 2000, 2200, 1800, 950, 1750, 3000, 1250]
    }
    df_sample = pd.DataFrame(data)
    df_sample.to_csv(filename, index=False)
    print(f"Sample data written to `{filename}`")

# 2. Graph analytics with NetworkX
def graph_analytics(df):
    G = nx.from_pandas_edgelist(df, 'Sender', 'Receiver', ['Amount'])
    centrality = nx.degree_centrality(G)
    return G, centrality

# 3. Charts & summary statistics
def generate_charts(df):
    os.makedirs("charts", exist_ok=True)
    os.makedirs("reports", exist_ok=True)

    sender_group = df.groupby('Sender')['Amount'].sum().reset_index()
    fig = px.bar(sender_group, x='Sender', y='Amount', title='Total Sent by Sender')
    fig.write_html("charts/sender_bar.html")

    cross_tab = pd.crosstab(df['Sender'], df['Receiver'], values=df['Amount'], aggfunc='sum', dropna=False)
    cross_tab.to_csv("reports/crosstab.csv")

    stats = df['Amount'].describe()
    stats.to_csv("reports/statistics.csv")

    return sender_group, stats, cross_tab

# 4. Filter, sort, group
def filter_sort_group(df):
    df_filtered = df[df['Amount'] > 1000]
    df_sorted = df_filtered.sort_values(by='Amount', ascending=False)
    df_grouped = df_sorted.groupby('Sender').agg({'Amount': ['sum', 'mean']}).reset_index()
    return df_grouped

# 5. Add summary line
def add_summary_line(df):
    total = df['Amount'].sum()
    return f"Total Transaction Value: ₹{total:.2f}"

# 6. Export reports
def export_reports(df, summary):
    # Flatten MultiIndex columns from groupby
    df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]

    df.to_excel("reports/report.xlsx", index=False)
    df.to_csv("reports/report.csv", index=False)
    df.to_xml("reports/report.xml", index=False)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Transactional Report", ln=True, align='C')
    pdf.ln(10)

    for _, row in df.iterrows():
        pdf.cell(200, 10, txt=str(row.to_dict()), ln=True)
        pdf.ln(5)

    pdf.set_font("Arial", style='B', size=12)
    pdf.cell(200, 10, txt=summary.encode('utf-8').decode('latin-1'), ln=True)
    pdf.output("reports/report.pdf")

# 7. Run everything
def run_report():
    create_sample_csv()
    df = pd.read_csv("transactions.csv")
    G, centrality = graph_analytics(df)
    sender_group, stats, crosstab = generate_charts(df)
    processed_data = filter_sort_group(df)
    summary = add_summary_line(df)
    export_reports(processed_data, summary)
    print("✅ Reports generated! Check the left file browser in Colab to download.")

# Run it
run_report()

Sample data written to `transactions.csv`
✅ Reports generated! Check the left file browser in Colab to download.


In [5]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
from fpdf import FPDF
import os

# 1. Load data from CSV
def load_data(filepath):
    df = pd.read_csv(filepath)
    print(f"Loaded data with {len(df)} records.")
    return df

# 2. Graph analytics with NetworkX
def graph_analytics(df, sender_col='Customer ID', receiver_col='Country', amount_col='Amount'):
    G = nx.from_pandas_edgelist(df, sender_col, receiver_col, [amount_col])
    centrality = nx.degree_centrality(G)
    return G, centrality

# 3. Generate charts & statistics
def generate_charts(df, sender_col='Customer ID', amount_col='Amount'):
    os.makedirs("charts", exist_ok=True)
    os.makedirs("reports", exist_ok=True)

    top_senders = df.groupby(sender_col)[amount_col].sum().reset_index().sort_values(by=amount_col, ascending=False).head(10)
    fig = px.bar(top_senders, x=sender_col, y=amount_col, title='Top 10 Customers by Spending')
    fig.write_html("charts/top_customers.html")

    stats = df[amount_col].describe()
    stats.to_csv("reports/statistics.csv")

    crosstab = pd.crosstab(df[sender_col], df['Country'], values=df[amount_col], aggfunc='sum', dropna=False)
    crosstab.to_csv("reports/crosstab.csv")

    return top_senders, stats, crosstab

# 4. Filter, sort, group
def filter_sort_group(df, sender_col='Customer ID', amount_col='Amount'):
    df_filtered = df[df[amount_col] > 1000]
    df_sorted = df_filtered.sort_values(by=amount_col, ascending=False)
    df_grouped = df_sorted.groupby(sender_col).agg(
        Total_Amount=pd.NamedAgg(column=amount_col, aggfunc='sum'),
        Average_Amount=pd.NamedAgg(column=amount_col, aggfunc='mean'),
        Transaction_Count=pd.NamedAgg(column=amount_col, aggfunc='count')
    ).reset_index()
    return df_grouped

# 5. Summary line
def add_summary_line(df, amount_col='Amount'):
    total = df[amount_col].sum()
    return f"Total Transaction Value: ₹{total:,.2f}"

# 6. Export reports
def export_reports(df, summary, filename_prefix="report"):
    df.to_excel(f"reports/{filename_prefix}.xlsx", index=False)
    df.to_csv(f"reports/{filename_prefix}.csv", index=False)

    # Sanitize column names for XML (remove spaces)
    df_xml = df.copy()
    df_xml.columns = [col.replace(" ", "_") for col in df.columns]
    df_xml.to_xml(f"reports/{filename_prefix}.xml", index=False)

    # Generate PDF report
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Transactional Report", ln=True, align='C')
    pdf.ln(10)

    for _, row in df.iterrows():
        line = ', '.join([f"{k}: {v}" for k, v in row.items()])
        pdf.multi_cell(0, 10, txt=line)
        pdf.ln(1)

    pdf.set_font("Arial", style='B', size=12)
    pdf.ln(5)
    pdf.cell(200, 10, txt=summary.encode('utf-8').decode('latin-1'), ln=True)
    pdf.output(f"reports/{filename_prefix}.pdf")

# 7. Main function to run everything
def run_report(filepath):
    df = load_data(filepath)

    # Graph Analytics
    G, centrality = graph_analytics(df)

    # Charts & Stats
    top_customers, stats, crosstab = generate_charts(df)

    # Filtered, Grouped data
    processed_data = filter_sort_group(df)

    # Summary
    summary = add_summary_line(df)

    # Export
    export_reports(processed_data, summary)
    print("✅ All reports generated successfully!")

# Run the report (replace with your actual file path)
run_report("data-analytics A6.csv")


Loaded data with 10000 records.
✅ All reports generated successfully!
